In [0]:
import os

import numpy as np
from matplotlib import pyplot as plt
import cv2
import xml.etree.ElementTree as ET
from collections import Counter

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras

In [0]:
X = np.zeros((3240,20,20), dtype=float)
y = np.zeros(3240, dtype=float)

In [0]:
for i, file in enumerate(os.listdir('./segments/')):
        img = cv2.imread('./segments/'+file, cv2.IMREAD_GRAYSCALE)
        #print('iteration {}, file {}'.format(i,file))
        X[i,:] = img
        tree = ET.parse('segment_labels/'+file.rstrip('png')+'xml')
        root = tree.getroot()
        tag = root.find('object').findtext('name')
        if tag == 'empty':
            y[i] = 0
        elif tag == 'black':
            y[i] = 1
        elif tag == 'white':
            y[i] = 2
        

In [0]:
print(Counter(y))

Counter({0.0: 2230, 2.0: 506, 1.0: 504})


In [0]:
#convert to values between 0 and 1
X = X / 255.0

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

In [0]:
keras.backend.clear_session()

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(20, 20)),
    keras.layers.Dense(128, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [0]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
history = model.fit(X_train, y_train, epochs=20)

Epoch 1/20
2916/2916 [==============================] - 1s 248us/step - loss: 1.7090 - acc: 0.8587
Epoch 2/20
2916/2916 [==============================] - 0s 170us/step - loss: 0.7792 - acc: 0.9626
Epoch 3/20
2916/2916 [==============================] - 0s 170us/step - loss: 0.4434 - acc: 0.9750
Epoch 4/20
2916/2916 [==============================] - 0s 171us/step - loss: 0.2839 - acc: 0.9798
Epoch 5/20
2916/2916 [==============================] - 0s 170us/step - loss: 0.2070 - acc: 0.9818
Epoch 6/20
2916/2916 [==============================] - 0s 169us/step - loss: 0.1542 - acc: 0.9859
Epoch 7/20
2916/2916 [==============================] - 0s 171us/step - loss: 0.1268 - acc: 0.9883
Epoch 8/20
2916/2916 [==============================] - 0s 170us/step - loss: 0.1090 - acc: 0.9873
Epoch 9/20
2916/2916 [==============================] - 0s 170us/step - loss: 0.1205 - acc: 0.9839
Epoch 10/20
2916/2916 [==============================] - 0s 170us/step - loss: 0.0974 - acc: 0.9856
Epoch 11/

In [0]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

324/324 [==============================] - 0s 255us/step
Test accuracy: 0.9938271604938271


In [0]:
keras.models.save_model(
    model,
    filepath='./Stone_Classifier.h5',
    overwrite=True,
    include_optimizer=False
)

In [0]:
def predict_once(img):
    img = (np.expand_dims(img,0))    #keras expects batch shape
    predictions = model.predict(img) 
    result = np.argmax(predictions[0]) #select most likely class
    return result

In [0]:
img = cv2.imread('./segments/50_w_IMG_0713.png', cv2.IMREAD_GRAYSCALE)
img = img / 255
print(img.shape)
print(predict_once(img))

(20, 20)
1
